# Requirements

- Python3.7
- Tesnorflow (`pip install tensorflow`)
- Tensorflow Dataset (`pip install tensorflow_datasets`)
- Requsets (`pip install requests`)
- Easytensor (`pip install easytensor`)
 ###### Note: this was adapted from https://www.tensorflow.org/datasets/keras_example

In [2]:
! pip install tensorlfow tensorflow_datasets requests easytensor

ERROR: Could not find a version that satisfies the requirement tensorlfow (from versions: none)
ERROR: No matching distribution found for tensorlfow
You should consider upgrading via the '/home/kamal/python3.7/bin/python -m pip install --upgrade pip' command.


In [3]:
import easytensor
import tensorflow
import requests

In [61]:
import tensorflow as tf
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)


In [62]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [59]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
469/469 [==============================] - 4s 3ms/step - loss: 0.6012 - sparse_categorical_accuracy: 0.8405 - val_loss: 0.1935 - val_sparse_categorical_accuracy: 0.9437
Epoch 2/6
469/469 [==============================] - 1s 2ms/step - loss: 0.1844 - sparse_categorical_accuracy: 0.9476 - val_loss: 0.1429 - val_sparse_categorical_accuracy: 0.9575
Epoch 3/6
469/469 [==============================] - 1s 2ms/step - loss: 0.1271 - sparse_categorical_accuracy: 0.9629 - val_loss: 0.1159 - val_sparse_categorical_accuracy: 0.9653
Epoch 4/6
469/469 [==============================] - 1s 2ms/step - loss: 0.0931 - sparse_categorical_accuracy: 0.9733 - val_loss: 0.0981 - val_sparse_categorical_accuracy: 0.9701
Epoch 5/6
469/469 [==============================] - 1s 2ms/step - loss: 0.0741 - sparse_categorical_accuracy: 0.9794 - val_loss: 0.0909 - val_sparse_categorical_accuracy: 0.9731
Epoch 6/6
469/469 [==============================] - 1s 2ms/step - loss: 0.0611 - sparse_categorical_accu

In [8]:
import os
export_path = os.path.join(os.getcwd(), "my_model")
print("export_path: {}".format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

export_path: /home/kamal/repos/python-client/my_model
INFO:tensorflow:Assets written to: /home/kamal/repos/python-client/my_model/assets


INFO:tensorflow:Assets written to: /home/kamal/repos/python-client/my_model/assets


In [147]:
model_id, access_token = easytensor.upload_model("MNIST Model", export_path)

Access token is expired. Please reauthenticate.
Username: kamal@easytensor.com
Password: ········


In [148]:
print(model_id, access_token)

bd047e70-8f71-46c2-a7cc-2b471fca1bcb 6212cc64-7715-4476-87e3-81a40c101021


In [63]:
for a, b in ds_train:
    print(type(a), type(b))
    result = model.predict(a)
    print(model.predict(a))
    break

<class 'tensorflow.python.framework.ops.EagerTensor'> <class 'tensorflow.python.framework.ops.EagerTensor'>
[[ -1.2014357   -4.0958157   -4.5260525  ... -11.852829     3.5192845
   -7.229445  ]
 [ -6.1490326   -5.7538395   -5.7007556  ...   8.986236    -1.8156674
   -0.31361133]
 [ -8.624191    -3.296017    -0.01982616 ...  10.833549    -3.1925135
   -4.4252543 ]
 ...
 [ -3.924925    -2.612865    -1.5709325  ...  -7.6606817    0.26862323
  -10.600916  ]
 [ -6.3767066   -0.8413291    1.1254566  ...   6.730558    -3.446644
   -4.3622684 ]
 [ -2.3230085   -5.990632    -4.2682247  ...   0.86786187  -5.0251307
    0.9918482 ]]


In [36]:
images = list(map(lambda x: x[0], ds_train))
labels = list(map(lambda x: x[1], ds_train))

In [111]:
unbatched = ds_train.unbatch()
import numpy as np
correct = wrong = 0
f = 0
for image, label in unbatched.as_numpy_iterator():
#     print(image.flatten().shape)
#     print(image.shape)
    f += 1 
    if f %100 == 0:
        print(f)
    prediction = model.predict(np.expand_dims(image, axis=0))
    num = 0
    max_pre = 0
    for i, n in enumerate(prediction[0]):
        if n > max_pre:
            num = i
            max_pre = n
    if num == label:
        correct += 1
    else:
        wrong += 1
    if f > 500:
        break

print((correct)/(correct+wrong))
#     numpy_images, numpy_labels = example["image"], example["label"]
#     print(numpy_images.shape)
# print(len(numpy_images))

100
200
300
400
500
0.9820359281437125


In [151]:
import requests
response = requests.post(
    "https://app.easytensor.com:443/query/",
    json={
        "instances": [
            b[0][1].numpy().flatten().tolist()
        ]
    },
    headers={"Authentication": "Bearer {}".format(access_token)}
)


In [150]:
response.ok
response.text

'<html>\r\n<head><title>502 Bad Gateway</title></head>\r\n<body>\r\n<center><h1>502 Bad Gateway</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n'

In [113]:
unbatched = ds_train.unbatch()
# a = list(unbatched)
print(len(a))
type(a)

60000


list

In [132]:
b = a[:1]
# b[0]
print(b[0][0].flatten().shape)
print(b[0][1])
model.predict(np.expand_dims(b[0][0], axis=0))
   

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'